## 01- ETL : CCBI Index

This part covers the extraction of Crypto Fear and Greed data, as well as CBBI index 
    - Crypto F&G : https://api.alternative.me/
    - CCBI index : https://colintalkscrypto.com/ 



---
### 0. Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path

import requests
import json

from functions_library.functions import DFinfo

# datetime manipulation for dataframes
from datetime import datetime


---
### 1. Data Loading

#### Get latest "Crypto Fear & Greed" (FNG) data 

In [2]:
# Download Fear & Greed JSON data
url = "https://api.alternative.me/fng/?limit=0&format=json"
response = requests.get(url)

if response.status_code == 200:
    with open("resources/fear_and_greed_index.json", "wb") as file:
        file.write(response.content)
    print("Fear & Greed JSON data downloaded successfully.")
else:
    print("Failed to download FNG data.")

# Read the JSON data from the file
fng_json = Path("resources/fear_and_greed_index.json")
with open('resources/fear_and_greed_index.json') as file:
    data = json.load(file)

# Extract the "data" section from the JSON
data = data['data']

# Convert the data to a DataFrame
fng_df = pd.DataFrame(data)


Fear & Greed JSON data downloaded successfully.


In [3]:
# Convert the "timestamp" column to datetime format
fng_df['timestamp'] = pd.to_datetime(fng_df['timestamp'], unit='s')

# Set the "timestamp" column as the index
fng_df.set_index('timestamp', inplace=True)

# Clean the dataframe by dropping the "time_until_update" column
fng_df.drop("time_until_update",axis=1,inplace=True)

DFinfo(fng_df)


/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_59237/1257052585.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  fng_df['timestamp'] = pd.to_datetime(fng_df['timestamp'], unit='s')


,value,value_classification
timestamp,,
2025-02-18,47,Neutral
2025-02-17,51,Neutral
2025-02-16,54,Neutral


,value,value_classification
timestamp,,
2018-02-03,40,Fear
2018-02-02,15,Extreme Fear
2018-02-01,30,Fear


#### Get latest CBBI index from Colin Talks Crypto : https://colintalkscrypto.com/cbbi/

In [4]:
# Download CBBI JSON data
url_CBBI = "https://colintalkscrypto.com/cbbi/data/latest.json"
r = requests.get(url_CBBI, headers={"User-Agent": "XY"})

if r.status_code == 200:
    with open("resources/CBBI_index.json", "wb") as file:
        file.write(r.content)
    print("CBBI JSON data downloaded successfully.")
else:
    print("Failed to download CBBI data.")
    print(r)

# Load the JSON data from the file
CBBI_json = Path("resources/CBBI_index.json")
CBBI_df = pd.read_json(CBBI_json)

CBBI_df = CBBI_df.reset_index(names='timestamp')

# Convert the 'timestamp' column to a DatetimeIndex
CBBI_df['timestamp'] = pd.to_datetime(CBBI_df['timestamp'], unit='s')
CBBI_df.set_index('timestamp', inplace=True)

# Resample the DataFrame to hourly frequency and forward fill missing values
CCBI_df = CBBI_df.resample('1H').ffill()

# Print 5 rows of resulting DataFrame
DFinfo(CBBI_df)


CBBI JSON data downloaded successfully.


/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_59237/2115972912.py:24: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  CCBI_df = CBBI_df.resample('1H').ffill()


,Price,PiCycle,RUPL,RHODL,Puell,2YMA,Trolololo,MVRV,ReserveRisk,Woobull,Confidence
timestamp,,,,,,,,,,,
2011-06-27,15.59,NaN,0.8554,0.7587,0.7151,0.6004,0.7594,0.5333,NaN,0.5952,0.6882
2011-06-28,17.01,NaN,0.8571,0.7662,0.7004,0.6174,0.7843,0.5372,0.8265,0.6142,0.7129
2011-06-29,16.93,NaN,0.8571,0.7654,0.7145,0.6120,0.7812,0.5378,0.8352,0.6089,0.7140


,Price,PiCycle,RUPL,RHODL,Puell,2YMA,Trolololo,MVRV,ReserveRisk,Woobull,Confidence
timestamp,,,,,,,,,,,
2025-02-15,97544.0000,0.6432,0.8664,1.0,0.8893,0.9465,0.5785,0.8163,0.6531,0.7057,0.7888
2025-02-16,97603.0000,0.6444,0.8666,1.0,0.8375,0.9460,0.5784,0.8151,0.6582,0.7059,0.7836
2025-02-17,96164.1038,0.6453,0.8582,1.0,0.8377,0.9353,0.5696,0.8215,0.6582,0.6964,0.7802


In [5]:
# def get_cnn_market_fng():
#     """
#     Get CNN Market Fear & Greed Index data from CNN Money
#     Returns a DataFrame with daily values
#     """
#     # CNN Money Fear & Greed API endpoint
#     url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
    
#     # Headers to mimic a browser request
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
#         'Accept': 'application/json',
#         'Accept-Language': 'en-US,en;q=0.9',
#         'Referer': 'https://www.cnn.com/markets/fear-and-greed',
#         'Origin': 'https://www.cnn.com'
#     }
    
#     try:
#         response = requests.get(url, headers=headers)
        
#         if response.status_code == 200:
#             # Save raw data
#             with open("resources/cnn_market_fear_and_greed.json", "wb") as file:
#                 file.write(response.content)
#             print("CNN Market Fear & Greed JSON data downloaded successfully.")
            
#             # Parse the JSON data
#             data = response.json()
            
#             # Extract the fear and greed data
#             fear_greed_data = []
#             for entry in data['fear_and_greed_historical']['data']:
#                 timestamp = pd.to_datetime(entry['x'], unit='ms')
#                 value = entry['y']
                
#                 # Get classification based on value
#                 if value >= 75:
#                     classification = "Extreme Greed"
#                 elif value >= 50:
#                     classification = "Greed"
#                 elif value >= 25:
#                     classification = "Fear"
#                 else:
#                     classification = "Extreme Fear"
                    
#                 fear_greed_data.append({
#                     'timestamp': timestamp,
#                     'value': value,
#                     'value_classification': classification
#                 })
            
#             # Convert to DataFrame
#             df = pd.DataFrame(fear_greed_data)
#             df.set_index('timestamp', inplace=True)
            
#             # Sort by timestamp
#             df = df.sort_index()
            
#             # Save processed data
#             df.to_csv('Resources/CNN_Market_FNG_1D_new.csv')
            
#             return df
            
#         else:
#             print(f"Failed to download CNN Market F&G data. Status code: {response.status_code}")
#             print(f"Response content: {response.text}")
#             return None
            
#     except Exception as e:
#         print(f"Error downloading CNN Market F&G data: {str(e)}")
#         return None

# # Get the data
# cnn_market_fng_df = get_cnn_market_fng()

# if cnn_market_fng_df is not None:
#     print("\nFirst few rows:")
#     print(cnn_market_fng_df.head())
#     print("\nLast few rows:")
#     print(cnn_market_fng_df.tail())

In [6]:
def download_and_merge_cnn_market_fng():
    """
    Download and merge CNN Market Fear & Greed Index data
    Handles existing CSV files and updates with new data
    """
    filename = 'CNN_Market_FNG_1D'
    csv_path = f'Resources/{filename}.csv'
    
    if Path(csv_path).is_file():
        # Load existing data
        result_df = pd.read_csv(csv_path, parse_dates=['timestamp'], index_col='timestamp')
        
        latest_timestamp = result_df.index.max()
        first_timestamp = result_df.index.min()
        
        print(f"Detecting CNN Market F&G data that was previously downloaded:")
        print(f" > latest timestamp available: {latest_timestamp}")
        print(f" > first timestamp available: {first_timestamp}")
        print("")
        
        # Get new data from CNN API
        url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'application/json',
            'Referer': 'https://www.cnn.com/markets/fear-and-greed'
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Save raw data
            with open("Resources/cnn_market_fear_and_greed.json", "wb") as file:
                file.write(response.content)
            
            data = response.json()
            new_data = []
            
            for entry in data['fear_and_greed_historical']['data']:
                timestamp = pd.to_datetime(entry['x'], unit='ms')
                if timestamp > latest_timestamp:
                    value = entry['y']
                    
                    # Get classification
                    if value >= 75:
                        classification = "Extreme Greed"
                    elif value >= 50:
                        classification = "Greed"
                    elif value >= 25:
                        classification = "Fear"
                    else:
                        classification = "Extreme Fear"
                    
                    new_data.append({
                        'timestamp': timestamp,
                        'value': value,
                        'value_classification': classification
                    })
            
            if new_data:
                # Convert new data to DataFrame
                new_df = pd.DataFrame(new_data)
                new_df.set_index('timestamp', inplace=True)
                
                # Combine existing and new data
                result_df = pd.concat([result_df, new_df])
                
                # Remove duplicates and sort
                result_df = result_df[~result_df.index.duplicated(keep='last')].sort_index()
                
                print(f"Added {len(new_data)} new entries")
            else:
                print("No new data to add")
                
    else:
        # If no existing file, get all available data
        url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'application/json',
            'Referer': 'https://www.cnn.com/markets/fear-and-greed'
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Save raw data
            with open("Resources/cnn_market_fear_and_greed.json", "wb") as file:
                file.write(response.content)
            
            data = response.json()
            fear_greed_data = []
            
            for entry in data['fear_and_greed_historical']['data']:
                timestamp = pd.to_datetime(entry['x'], unit='ms')
                value = entry['y']
                
                # Get classification
                if value >= 75:
                    classification = "Extreme Greed"
                elif value >= 50:
                    classification = "Greed"
                elif value >= 25:
                    classification = "Fear"
                else:
                    classification = "Extreme Fear"
                
                fear_greed_data.append({
                    'timestamp': timestamp,
                    'value': value,
                    'value_classification': classification
                })
            
            # Convert to DataFrame
            result_df = pd.DataFrame(fear_greed_data)
            result_df.set_index('timestamp', inplace=True)
            result_df = result_df.sort_index()
    
    # Save the updated data
    result_df.to_csv(csv_path)
    print(f"\nCNN Market F&G data has been saved to disk and is available now =)")
    print("\nFirst few rows:")
    print(result_df.head())
    print("\nLast few rows:")
    print(result_df.tail())
    
    return result_df

def download_and_merge_cbbi():
    """
    Download and merge CBBI (Colin Talks Crypto Bitcoin Bull Run Index) data
    Handles existing CSV files and updates with new data
    """
    filename = 'CBBI_index'
    csv_path = f'Resources/{filename}.csv'
    
    if Path(csv_path).is_file():
        # Load existing data
        result_df = pd.read_csv(csv_path, parse_dates=['timestamp'], index_col='timestamp')
        
        latest_timestamp = result_df.index.max()
        first_timestamp = result_df.index.min()
        
        print(f"Detecting CBBI data that was previously downloaded:")
        print(f" > latest timestamp available: {latest_timestamp}")
        print(f" > first timestamp available: {first_timestamp}")
        print("")
        
        # Get new data
        url_CBBI = "https://colintalkscrypto.com/cbbi/data/latest.json"
        response = requests.get(url_CBBI, headers={"User-Agent": "XY"})
        
        if response.status_code == 200:
            # Save raw data
            with open("Resources/CBBI_index.json", "wb") as file:
                file.write(response.content)
            
            # Parse JSON from response content
            new_df = pd.read_json(response.content.decode('utf-8'))
            new_df = new_df.reset_index(names='timestamp')
            new_df['timestamp'] = pd.to_datetime(new_df['timestamp'], unit='s')
            new_df.set_index('timestamp', inplace=True)
            
            # Only keep new data
            new_df = new_df[new_df.index > latest_timestamp]
            
            if not new_df.empty:
                # Combine existing and new data
                result_df = pd.concat([result_df, new_df])
                
                # Remove duplicates and sort
                result_df = result_df[~result_df.index.duplicated(keep='last')].sort_index()
                
                print(f"Added {len(new_df)} new entries")
            else:
                print("No new data to add")
                
    else:
        # If no existing file, get all available data
        url_CBBI = "https://colintalkscrypto.com/cbbi/data/latest.json"
        response = requests.get(url_CBBI, headers={"User-Agent": "XY"})
        
        if response.status_code == 200:
            # Save raw data
            with open("Resources/CBBI_index.json", "wb") as file:
                file.write(response.content)
            
            # Parse JSON from response content
            result_df = pd.read_json(response.content.decode('utf-8'))
            result_df = result_df.reset_index(names='timestamp')
            result_df['timestamp'] = pd.to_datetime(result_df['timestamp'], unit='s')
            result_df.set_index('timestamp', inplace=True)
    
    # Save the updated data
    result_df.to_csv(csv_path)
    print(f"\nCBBI data has been saved to disk and is available now =)")
    print("\nFirst few rows:")
    print(result_df.head())
    print("\nLast few rows:")
    print(result_df.tail())
    
    return result_df

# Download and update both indices
cnn_fng_df = download_and_merge_cnn_market_fng()
cbbi_df = download_and_merge_cbbi()

Detecting CNN Market F&G data that was previously downloaded:
 > latest timestamp available: 2024-12-03 15:56:52
 > first timestamp available: 2021-02-01 00:00:00

Added 53 new entries

CNN Market F&G data has been saved to disk and is available now =)

First few rows:
            value value_classification
timestamp                             
2021-02-01   39.0                 Fear
2021-02-02   58.0                Greed
2021-02-03   50.0                Greed
2021-02-04   60.0                Greed
2021-02-05   60.0                Greed

Last few rows:
                         value value_classification
timestamp                                          
2025-02-11 00:00:00  43.542857                 Fear
2025-02-12 00:00:00  42.971429                 Fear
2025-02-13 00:00:00  43.314286                 Fear
2025-02-14 00:00:00  44.485714                 Fear
2025-02-14 23:59:56  44.485714                 Fear
Detecting CBBI data that was previously downloaded:
 > latest timestamp avail

/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_59237/2129380293.py:159: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  new_df = pd.read_json(response.content.decode('utf-8'))
